In [ ]:
import re

import hvplot.pandas
import hvplot.xarray
import numpy as np
import pandas as pd
import pypsa
import seaborn
import xarray as xr

seaborn.set_style("whitegrid")

n = pypsa.Network()

n.import_from_netcdf(snakemake.input["network"])

## +++  Costs for electricity generators
df = n.generators.reset_index()

# Attribute storage and generator costs for CSP also toward electricity generation
df_s = n.stores.filter(like="csp-tower", axis="index").reset_index()
df_s["carrier"] = "csp-tower"
df_s = df_s.rename(columns={"e_nom_opt": "p_nom_opt"})

df_l = n.links.filter(like="csp-tower", axis="index").reset_index()
df_l["carrier"] = "csp-tower"

df = pd.concat([df, df_s, df_l])

df["annual cost"] = df["p_nom_opt"] * df["capital_cost"]

df = df.groupby("carrier")["annual cost"].sum()
df_generation_cost = df
df_generation_cost.name = "Total cost"

## +++  Electricity generation by source (generator)
df = n.generators_t["p"]

# csp-tower electricity output is from turbine (link), not from generator
df = df.filter(regex="^(?!csp-tower)", axis="columns")

df = df.sum(axis="index")

# Get electricity output from csp turbines
df_l = n.links_t["p1"].filter(like="csp-tower", axis="columns")
df_l = df_l.sum(axis="index") * (-1)
df_l.index = df_l.index.str.replace(
    " power block", ""
)  # rename for technology name instead link name

df = pd.concat([df, df_l])

df.index = df.index.str.replace(
    "\d", "", regex=True
).str.strip()  # Remove digits from different components and spaces

df = df.groupby(df.index).sum()

df_electricity_generation = df

# LCoE per RES tech
df_lcoe = df_generation_cost / df_electricity_generation
df_lcoe.name = "LCoE"

## +++  Installed capacity for electricity generation
df = n.generators["p_nom_opt"]

# csp-tower electricity output is from turbine (link), not from generator
df = df.filter(regex="^(?!csp-tower)", axis="index")

# Get electricity generation capacity from csp turbines
df_l = (n.links["p_nom_opt"] * n.links["efficiency"]).filter(
    like="csp-tower", axis="index"
)

df = pd.concat([df, df_l])

df.index = df.index.str.replace(
    "\d", "", regex=True
).str.strip()  # Remove digits from different components and spaces

df = df.groupby(df.index).sum()

df_electricity_capacity = df

## +++  Available potentials vs capacities for RES source (i.e. CSP: heat generation, not electricity)
df = n.generators.groupby("carrier")[["p_nom_max", "p_nom_opt"]].sum()
df = df.rename(columns={"p_nom_opt": "installed", "p_nom_max": "potential"})

df_res_capacities = df

## +++  Realised capacity factors based on generated output per RES tech
# Rename csp-tower power block to align with index from elec gen.
df_real_cf = (
    df_electricity_generation
    / (
        df_electricity_capacity.rename({"csp-tower power block": "csp-tower"})
        * len(n.snapshots)
    )
    * 100
)

## +++  Calculate capacity factors by class and technology based on output series
df = n.generators_t["p"] / n.generators["p_nom_opt"]
df = df.mean(axis="index")

# All non-csp technologies
df = df.filter(regex="^(?!csp-tower)", axis="index")

# Special treatment for CSP: Use turbine output as indicator
df_l = n.links_t["p1"] / (n.links["p_nom_opt"] * n.links["efficiency"])
df_l = df_l.mean(axis="index") * (-1)
df_l = df_l.filter(like="csp-tower", axis="index")
df_l.index = df_l.index.str.replace(" power block", "")

df = pd.concat([df, df_l])
df.name = "capacity factor"

df = df.to_frame()
df["technology"] = df.index.str.replace("\s\d+", "")
df["class"] = df.index.str.replace("\D+\s", "").astype(int)
df["capacity factor"] *= 100
df_classtech_cf = df

## +++ Pre-Plot individual figures
fig_generation_cost = df_generation_cost.hvplot.bar(
    title="Generation cost",
    xlabel="technology",
    ylabel="Annualised cost [EUR]",
    yaxis="left",
    shared_axes=False,
)
fig_lcoe = df_lcoe.hvplot.bar(
    xlabel="technology", ylabel="LCoE [EUR/MWh]", shared_axes=False
)
fig_electricity_generation = df_electricity_generation.hvplot.bar(
    title="Electricity generation",
    xlabel="technology",
    ylabel="Annual generation [MWh]",
    shared_axes=False,
)
fig_res_capacities = df_res_capacities.hvplot.bar(
    title="RES: Potential vs. installed capacities",
    xlabel="technology",
    ylabel="Capacity [MW]",
    rot=45,
    shared_axes=False,
)
fig_real_cf = df_real_cf.hvplot.bar(
    title="Capacity factors (weighted by output)",
    xlabel="technology",
    ylabel="Annual capacity factor [%]",
    shared_axes=False,
)
fig_classtech_cf = df_classtech_cf.hvplot(
    title="Capacity factors by class (based on output and capacity)",
    ylabel="Annual capacity factor [%]",
    by="technology",
    x="class",
    y="capacity factor",
    shared_axes=False,
)
fig_electricity_capacity = df_electricity_capacity.hvplot.bar(
    title="Installed electric capacity",
    xlabel="technology",
    ylabel="Peak capacity [MW]",
    shared_axes=False,
)

## +++ Combine figures to one big figure
fig = (
    fig_generation_cost
    + fig_lcoe
    + fig_electricity_generation
    + fig_res_capacities
    + fig_real_cf
    + fig_classtech_cf
    + fig_electricity_capacity
).cols(2)

## +++ Save figures
for fp in snakemake.output["fig"]:
    hvplot.save(fig, fp)